In [4]:
from cryptography.hazmat.primitives import hashes, serialization
from cryptography.hazmat.primitives.asymmetric import rsa, padding
import base64

# ============================
# Generate RSA key pair
# ============================
def generate_rsa_keys():
    private_key = rsa.generate_private_key(
        public_exponent=65537,    # Standard and secure
        key_size=15360
    )
    public_key = private_key.public_key()

    # Serialize keys to PEM format (for saving or sharing)
    private_pem = private_key.private_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PrivateFormat.PKCS8,
        encryption_algorithm=serialization.NoEncryption()
    )
    public_pem = public_key.public_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PublicFormat.SubjectPublicKeyInfo
    )

    return private_key, public_key, private_pem.decode(), public_pem.decode()

# ============================
# Encrypt with public key
# ============================
def encrypt_rsa(message: str, public_key) -> str:
    ciphertext = public_key.encrypt(
        message.encode('utf-8'),
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )
    return base64.b64encode(ciphertext).decode('utf-8')

# ============================
# Decrypt with private key
# ============================
def decrypt_rsa(encrypted_b64: str, private_key) -> str:
    ciphertext = base64.b64decode(encrypted_b64)
    plaintext = private_key.decrypt(
        ciphertext,
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )
    return plaintext.decode('utf-8')

# ============================
# Demo
# ============================
if __name__ == "__main__":
    print("Generating RSA key pair...\n")
    private_key, public_key, private_pem, public_pem = generate_rsa_keys()
    
    print(f"RSA Modulus Strength: {public_key.key_size} bits\n")

    message = "Hello from RSA! This message is confidential. 🔒"

    print(f"Original message: {message}\n")

    # Encrypt with public key
    encrypted = encrypt_rsa(message, public_key)
    print("Encrypted (Base64):")
    print(encrypted)
    print()

    # Decrypt with private key
    decrypted = decrypt_rsa(encrypted, private_key)
    print(f"Decrypted message: {decrypted}\n")

    # Verify
    assert decrypted == message
    print("Success! RSA encryption and decryption worked perfectly.")

    # Optional: Show public key (safe to share)
    print("\nPublic Key (can be shared openly):")
    print(public_pem)

Generating RSA key pair...

RSA Modulus Strength: 15360 bits

Original message: Hello from RSA! This message is confidential. 🔒

Encrypted (Base64):
FPlmyyCwly9XZA7mkXeL5/8agVSHwIWAVLxrNhEAPl8R3k3iEAzuxUwzhc9CGG5M+4Q8mt9XCzIkqeIjnwyH+brqbZxTPI6Sd/QD4rfFpo4I/2dw1AqBnGFdPVwDHGCf30MGdG76+P4nE9MQhx54N6i0ojVxKPn33seWLj8bACdvy14WninhLkO6xTTfYF0SVCpkc97ZXGZCDwYLcF59sZtmFhHMMqJLamofWSbJRwRbk1/sMZTYtf8Y96ml2aYHcjFGoSrdx58sv+OKewhg+56aA/pCGO7hIGLesYW5lKkN0m1AenVpcUZ5lb8V7GdUp8lY4kfrlzl7Au7QhaIde0gRIaWtZN4LiW5wWislE3JqKV4Rq24BdbgbLgUVkBhaYFQUUnQDAQcYJMXshOz5fQMkZgdzkDKPOvzoQ22D5yjh/NlgjDEALgqdtMOSaB55VTp7mxPTQ3UK33C9lKIpaCWIxrY7WRwMixKABRSE6rjofboCFtV1gONizUlpcCPkrfK55lSQIEMyKP6tSsa1S9fUBRFtbmT/b86t4wY86aRuPoWMMTYAALpSwzmxNPNOtoNTrB0lRYCWez9JFgAjWOYJzghXY+kYlRrjzf7joY31Rc0gqIWe/21VM1qh4FSw1lsKd4jnp7MVW861Nd5mjfcJpAC+AUGx9Vcq8pifXG4z+cUqn1qa//zuDSLdJijGygvkPzTX/Y1d4PjhpJG7+SVWorj/NJqnzfLHq+Pc/VEdyTGQxzFcDTDlr8UujfGEaiAqmIkcVfeM4DK0ctzDql1KxqhJWHsxFHp8emsletbmuW2Ca322d8z+nzqUnm3XjuT5M1uVPMYESZHiqyH